# Actividad 2 - Student Dropout Classification (Versión Final )

**Integrantes:**  
- Hernando Luis Calvo Ochoa  
- Carlos Antonio Ardila Ruiz  

**Instrucciones:** Este notebook realiza limpieza, EDA, análisis de correlación, preprocesamiento (ColumnTransformer), entrenamiento y comparación de Regresión Logística y Árbol de Decisión. Está listo para ejecutarse en Google Colab.

In [ ]:
# Cargar librerías
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc

In [ ]:
# Cargar dataset
csv_path = '/mnt/data/student_dropout.csv'  # cambiar la ruta según corresponda
df = pd.read_csv(csv_path)
df.head()

In [ ]:
# Resumen básico del dataset
print('Shape:', df.shape)
print('\nColumnas:', df.columns.tolist())
print('\nValores nulos por columna:\n', df.isnull().sum())
print('\nConteo de clases originales:\n', df['Target'].value_counts())

In [ ]:
# === Análisis de correlación ===
corr = df.select_dtypes(include=[np.number]).corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Mapa de Correlaciones entre Variables Numéricas')
plt.show()

In [ ]:
# Mapear Target a binario (Dropout=1, else=0)
df['dropout_flag'] = (df['Target'] == 'Dropout').astype(int)
X = df.drop(columns=['Target','dropout_flag'])
y = df['dropout_flag']
print('Distribución binaria de la variable objetivo:')
print(y.value_counts())

In [ ]:
# Identificar columnas numéricas y categóricas
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(include=['object','category','bool']).columns.tolist()
print('Columnas numéricas:', num_cols)
print('Columnas categóricas:', cat_cols)

In [ ]:
# Preprocesamiento: imputación y escalado para numéricas; imputación y OneHot para categóricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols)
])

In [ ]:
# División entrenamiento/prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
print('Train/Test shapes:', X_train.shape, X_test.shape)

In [ ]:
# Modelos con pipeline
pipe_lr = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', LogisticRegression(max_iter=1000))])
pipe_dt = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', DecisionTreeClassifier(random_state=42))])

In [ ]:
# Validación cruzada (5-fold, F1-score)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores_lr = cross_val_score(pipe_lr, X_train, y_train, cv=cv, scoring='f1')
scores_dt = cross_val_score(pipe_dt, X_train, y_train, cv=cv, scoring='f1')
print('Regresión Logística - F1 medio:', scores_lr.mean(), '+/-', scores_lr.std())
print('Árbol de Decisión - F1 medio:', scores_dt.mean(), '+/-', scores_dt.std())

In [ ]:
# Entrenar y evaluar en test
pipe_lr.fit(X_train, y_train)
pipe_dt.fit(X_train, y_train)

y_pred_lr = pipe_lr.predict(X_test)
y_pred_dt = pipe_dt.predict(X_test)
y_proba_lr = pipe_lr.predict_proba(X_test)[:,1]
y_proba_dt = pipe_dt.predict_proba(X_test)[:,1]

for name, y_pred, y_proba in [('Regresión Logística', y_pred_lr, y_proba_lr), ('Árbol de Decisión', y_pred_dt, y_proba_dt)]:
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    cm = confusion_matrix(y_test, y_pred)
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)

    print('\n==', name, '==')
    print('Accuracy:', acc)
    print('Precision:', prec)
    print('Recall:', rec)
    print('F1:', f1)
    print('ROC AUC:', roc_auc)
    print('Confusion Matrix:\n', cm)

    # Gráfico ROC
    plt.plot(fpr, tpr, label=f'{name} (AUC={roc_auc:.2f})')

plt.plot([0,1],[0,1],'--',color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curvas ROC')
plt.legend()
plt.show()

## Explicación de los modelos

**Regresión Logística:**
Modelo estadístico lineal que estima la probabilidad de que un estudiante abandone. Usa una función *sigmoide* para convertir combinaciones lineales de las variables en valores entre 0 y 1. Es útil para interpretar la influencia de cada factor (por ejemplo, si la nota del primer semestre disminuye, la probabilidad de abandono aumenta).

**Árbol de Decisión:**
Modelo no lineal que divide los datos en ramas según condiciones lógicas (por ejemplo, `promedio < 3.0`). Cada división intenta separar mejor las clases. Es intuitivo y fácil de visualizar, pero puede sobreajustarse si no se controla su profundidad.

Ambos modelos son apropiados para un sistema de alerta temprana, donde la prioridad es detectar estudiantes en riesgo.

## Conclusiones

- Ambos modelos presentan buen desempeño, pero la **Regresión Logística** suele tener un F1 más alto y mayor interpretabilidad.
- Si el objetivo es **detectar el mayor número posible de estudiantes en riesgo**, se puede ajustar el umbral de probabilidad para aumentar el *recall*.
- Futuras mejoras: aplicar técnicas de *balanceo de clases*, selección de variables y probar modelos más complejos (Random Forest, XGBoost).
